In [ ]:
!pip install torch torchvision --quiet

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
seed = 42

# -----------------------
# Transform 정의
# -----------------------
train_transform = transforms.Compose([
    transforms.Resize(224),
    # =======================
    # Option 1: Crop
    # -----------------------
    transforms.Resize(256),
    transforms.RandomResizedCrop(224),
    # =======================
    # =======================
    # Option 2: Flip
    # -----------------------
    transforms.RandomHorizontalFlip(),
    # =======================
    # =======================
    # Option 3: ColorJitter 밝기·대비·채도·색조 변화
    # -----------------------
    # transforms.ColorJitter(
    # brightness=0.2,
    # contrast=0.2,
    # saturation=0.2,
    # hue=0.02
    # )
    # =======================
    # =======================
    # Option 4: Rotation
    # -----------------------
    transforms.RandomRotation(10),
    # =======================
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5)),
])

valid_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5)),
])

full_trainset = torchvision.datasets.CIFAR100(
    root="./data",
    train=True,
    download=True,
    transform=train_transform,
)

train_size = int(0.9 * len(full_trainset))
val_size = len(full_trainset) - train_size

trainset, validset = torch.utils.data.random_split(
    full_trainset,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(seed)
)

# Validation transform 적용
validset.dataset.transform = valid_transform


cuda


100%|██████████| 169M/169M [00:13<00:00, 12.3MB/s]


In [ ]:
class MyNet(nn.Module):
    def __init__(self, num_classes=10):
        super(MyNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 11, stride=4, padding=2),
            nn.ReLU(True),
            nn.MaxPool2d(3, stride=2),

            nn.Conv2d(64, 192, 5, padding=2),
            nn.ReLU(True),
            nn.MaxPool2d(3, stride=2),

            nn.Conv2d(192, 384, 3, padding=1),
            nn.ReLU(True),
            nn.Conv2d(384, 256, 3, padding=1),
            nn.ReLU(True),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)


In [ ]:
from torchvision.transforms.v2 import CutMix

cutmix = CutMix(alpha=1.0, num_classes=100)

In [ ]:
def train(model, loader, optimizer, criterion, use_cutmix=False):
    model.train()
    loss_sum, correct, total = 0, 0, 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        # ----------------------
        #   🔥 CutMix 적용
        # ----------------------
        if use_cutmix:
            images, labels = cutmix(images, labels)
            # CutMix에서는 labels가 one-hot이므로 loss 계산이 바뀜
            outputs = model(images)
            loss = (-labels * torch.log_softmax(outputs, dim=1)).sum(dim=1).mean()
        else:
            outputs = model(images)
            loss = criterion(outputs, labels)

        # ----------------------
        #   Backprop
        # ----------------------
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()

        # ----------------------
        #   Accuracy 계산
        #   (CutMix는 hard label로 평가하는 것이 일반적)
        # ----------------------
        _, predicted = outputs.max(1)
        total += labels.size(0)

        if use_cutmix:
            # One-hot to hard label for accuracy
            labels_hard = labels.argmax(dim=1)
            correct += predicted.eq(labels_hard).sum().item()
        else:
            correct += predicted.eq(labels).sum().item()

    return loss_sum / len(loader), 100 * correct / total

def test(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return 100 * correct / total


In [ ]:
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
validloader = DataLoader(validset, batch_size=64, shuffle=False)

In [ ]:
class MyNet(nn.Module):
    def __init__(self, num_classes=100):
        super(MyNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 11, stride=4, padding=2),
            nn.ReLU(True),
            nn.MaxPool2d(3, stride=2),

            nn.Conv2d(64, 192, 5, padding=2),
            nn.ReLU(True),
            nn.MaxPool2d(3, stride=2),

            nn.Conv2d(192, 384, 3, padding=1),
            nn.ReLU(True),
            nn.Conv2d(384, 256, 3, padding=1),
            nn.ReLU(True),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)

In [ ]:
model = MyNet().to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-7)
# >>>> lr scheduler 쓸 거라면, Option 1-3 중 "하나만" 주석 풀기 <<<<
# =======================
# Option 1: StepLR (일정 epoch마다 lr 감소)
# -----------------------
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
# =======================
# =======================
# Option 2: MultiStepLR (특정 epoch에서만 감소)
# -----------------------
# scheduler = optim.lr_scheduler.MultiStepLR(
#     optimizer,
#     milestones=[10, 14, 18],  # 이 epoch에서 lr 감소
#     gamma=0.1
# )
# =======================
# Option 3: Cosine Annealing (ResNet, ViT 등 현대 모델에서 추천됨)
# -----------------------
# scheduler = optim.lr_scheduler.CosineAnnealingLR(
#     optimizer,
#     T_max=20,   # 전체 epoch 수
# )
# =======================

best_val_acc = 0.0

for epoch in range(20):
    loss, train_acc = train(model, trainloader, optimizer, criterion,
                            use_cutmix=False)
    val_acc = test(model, validloader)

    # 만약 lr scheduler 쓸 거라면 주석 풀기
    # scheduler.step()

    print(f"[Epoch {epoch+1}] Loss: {loss:.4f} | Train Acc: {train_acc:.2f}% | Valid Acc: {val_acc:.2f}%")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "my_model.pt")
        print(f"  --> Saved best model (Val Acc = {val_acc:.2f}%)")


print("Training finished.")
print("Best model saved to my_model.pt")



KeyboardInterrupt: 